# Gallery Example: Single-Class Closed Queueing Network

This example creates a parameterizable closed queueing network with:
- **M stations**: Variable number of queues (default 2)
- **Single class**: One job class circulating in the network
- **Random population**: Random number of jobs (2 to M+2)
- **Random service rates**: Different service rates at each station
- **Optional delay**: Can include a delay station

This model is useful for testing multi-station closed network solvers.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_cqn(M=2, useDelay=false, seed=23000): Network {    """Create single-class closed queueing network        Parameters:    - M: Number of queue stations (default 2)    - useDelay: Whether to include delay station (default false)    - seed: Random seed for reproducibility (default 23000)    """    model = Network("Single-class CQN")        # Set random seed    random.seed(seed)    np.random.seed(seed)        # Block 1: nodes    station = []        for i in range(M):        queue = Queue(model, f"Queue{i+1}", SchedStrategy.PS)        station.append(queue)        if useDelay:        delay = Delay(model, "Delay1")        station.append(delay)        # Block 2: classes    population = int(random.random() * M + 2)  # Random population from 2 to M+2    jobclass = ClosedClass(model, "Class1", population, station[0], 0)        # Set service rates    for i in range(M):        service_rate = random.random() + i + 1  # Random rate between i+1 and i+2        station[i].setService(jobclass, Exp.fitMean(service_rate))        if useDelay:        station[-1].setService(jobclass, Exp.fitMean(2.0))        # Block 3: topology - serial routing    P = model.initRoutingMatrix()        # Create cyclic routing: station[0] -> station[1] -> ... -> station[0]    for i in range(len(station)):        next_station = station[(i + 1) % len(station)]        P.addRoute(jobclass, station[i], next_station, 1.0)        model.link(P)        return model, population# Create the modelmodel, population = gallery_cqn(M=3, useDelay=true, seed=23000)println(f"Population: {population} jobs")

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver (exact for closed models)
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver
val solver_ctmc = CTMC(model)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// NC Solver
val solver_nc = NC(model)
val avg_table_nc = solver_nc.avgTable
println("\nNC Solver:")
println(avg_table_nc)